In [ ]:
!git clone https://github.com/Seung-hwanSong/Anomaly.git #코랩 사용 

# [시계열 이상치 탐지 Part 2]
## Autoencoder

##### jupyter notebook 단축키

- ctrl+enter: 셀 실행   
- shift+enter: 셀 실행 및 다음 셀 이동   
- alt+enter: 셀 실행, 다음 셀 이동, 새로운 셀 생성
- a: 상단에 새로운 셀 만들기
- b: 하단에 새로운 셀 만들기
- dd: 셀 삭제(x: 셀 삭제)
- 함수 ( ) 안에서 shift+tab: arguments description. shift+tab+tab은 길게 볼 수 있도록

## 1. 모듈 불러오기

In [ ]:
''' 데이터 전처리 패키지 '''
import numpy as np
import pandas as pd

''' 기계학습 모델 구축 및 평가 패키지 '''
from sklearn.model_selection import train_test_split

''' 데이터 시각화 패키지 '''
import seaborn as sns
import matplotlib.pyplot as plt

## 2. 데이터 불러오기: NASA Bearing Dataset

- 데이터 description <br>
    - NASA Bearing Dataset은 NSF I/UCR Center의 Intelligent Maintenance System의 4개의 bearing에서 고장이 발생할 때까지 10분 단위로 수집된 센서 데이터이다. 본 데이터셋은 특정 구간에서 기록된 1-second vibration signal snapshots을 나타내는 여러 개의 파일로 구성되어 있다. 각 파일은 20 kHz 단위로 샘플링 된 20,480개의 data point를 포함하고 있으며, 각 파일의 이름은 데이터가 수집된 시간을 의미한다. 해당 데이터셋은 크게 3개의 데이터를 포함하고 있으며, 본 실습에서 사용하는 데이터는 outer race failure가 발생할 때까지 수집된 센서 데이터이다. <br><br>
- 변수
    - 센서 데이터: Bearing1, Bearing2, Bearing3, Bearing4 <br><br>
- 출처: https://ti.arc.nasa.gov/tech/dash/groups/pcoe/prognostic-data-repository/

<img src="https://i.imgur.com/dfFzn3H.jpg" width="600">

### Step1. 데이터 불러오기

In [ ]:
# 데이터 불러오기
data = pd.read_csv('/content/Anomaly/data/nasa_bearing_dataset.csv', index_col=0)
# data = pd.read_csv('./data/nasa_bearing_dataset.csv', index_col=0)

data.index = pd.to_datetime(data.index)
data.head()

In [ ]:
# 데이터 크기
print("데이터 크기 : ", data.shape)

### Step2. 데이터 Split

In [ ]:
X_train = data[data['data_type'] == 'train'].iloc[:, :4]

X_test = data[data['data_type'] == 'test'].iloc[:, :4]
y_test = data[data['data_type'] == 'test'].iloc[:, -2].values

In [ ]:
X_valid, X_test, y_valid, y_test = train_test_split(X_test, y_test, test_size=0.6, random_state=2024)

print("Training data shape:", X_train.shape)
print("Validation data shape:", X_valid.shape)
print("Test data shape:", X_test.shape)

## 3. Autoencoder
### (Auto-Associative Neural Network)

>입력과 출력이 동일한 인공신경망 구조 <br>
>정상 데이터에 대한 학습이 충분히 되어 있을 경우 정상 데이터는 자기 자신을 잘 복원할 수 있지만, 이상치 제이터는 학습 기회가 적어 상대적으로 잘 복원하지 못할 것을 가정

<img src="https://i.imgur.com/DTgug9o.jpg" width="700">

In [ ]:
# Autoencdoer 사용을 위한 torch 모듈 불러오기

import copy
import random

from sklearn.metrics import confusion_matrix, f1_score, roc_curve, auc
from sklearn.preprocessing import MinMaxScaler

import torch
import torch.nn as nn
import torchvision

In [ ]:
# hyper-parameter 설정
input_size = 4
hidden_size = 2

num_epochs = 500
batch_size = 32
learning_rate = 0.0001

random_seed = 2024
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") # Detect if we have a GPU available

In [ ]:
# seed 고정
torch.manual_seed(random_seed)
torch.cuda.manual_seed(random_seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(random_seed)
random.seed(random_seed)

### Step 0. 데이터 정규화

In [ ]:
# train 데이터를 기반으로 train/validation/test 데이터에 대하여 min-max scaling 적용 
scaler = MinMaxScaler()
scaler = scaler.fit(X_train)

In [ ]:
ae_X_train = scaler.transform(X_train)
ae_X_valid = scaler.transform(X_valid)
ae_X_test = scaler.transform(X_test)

In [ ]:
# train/test dataloader 생성
ae_train_dataset = torch.utils.data.TensorDataset(torch.Tensor(ae_X_train))
ae_train_loader = torch.utils.data.DataLoader(ae_train_dataset, batch_size=batch_size, shuffle=True)

ae_test_dataset = torch.utils.data.TensorDataset(torch.Tensor(ae_X_test))
ae_test_loader = torch.utils.data.DataLoader(ae_test_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
# validation dataloader의 경우 train/test 단계에서 사용될 loader를 따로 생성함
# train 단계: 정상으로만 구성, test 단계: 정상과 비정상으로 구성
ae_X_valid_train = ae_X_valid[y_valid == 0]
ae_valid_train_dataset = torch.utils.data.TensorDataset(torch.Tensor(ae_X_valid_train))
ae_valid_train_loader = torch.utils.data.DataLoader(ae_valid_train_dataset, batch_size=batch_size, shuffle=False)

ae_valid_test_dataset = torch.utils.data.TensorDataset(torch.Tensor(ae_X_valid))
ae_valid_test_loader = torch.utils.data.DataLoader(ae_valid_test_dataset, batch_size=batch_size, shuffle=False)

### Step 2. 모델 구축

In [ ]:
class AutoEncoder(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(AutoEncoder, self).__init__()
        self.encoder = nn.Sequential(
          nn.Linear(input_size, hidden_size),
          nn.ReLU()
        )
        self.decoder = nn.Sequential(
          nn.Linear(hidden_size, input_size)
        )
        
    def forward(self, x):
        out = self.encoder(x)
        out = self.decoder(out)
        out = torch.sigmoid(out)
        return out

In [ ]:
ae_model = AutoEncoder(input_size, hidden_size)

In [ ]:
ae_model

### Step 3. Train 데이터로 모델 학습

In [ ]:
# model 학습
def train_model(dataloaders, model, criterion, num_epochs, learning_rate, device):
    model = model.to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    
    best_model_wts = copy.deepcopy(model.state_dict())
    best_valid_loss = 10000000

    for epoch in range(num_epochs):
        if epoch == 0 or (epoch + 1) % 50 == 0:
            print()
            print('Epoch {}/{}'.format(epoch + 1, num_epochs))

        # 각 epoch마다 순서대로 training과 validation을 진행
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # 모델을 training mode로 설정
            else:
                model.eval()   # 모델을 validation mode로 설정

            running_loss = 0.0
            running_total = 0

            # training과 validation 단계에 맞는 dataloader에 대하여 학습/검증 진행
            for i, inputs in enumerate(dataloaders[phase]):
                inputs = inputs[0].to(device)

                # parameter gradients를 0으로 설정
                optimizer.zero_grad()

                # forward
                # training 단계에서만 gradient 업데이트 수행
                with torch.set_grad_enabled(phase == 'train'):
                    # input을 model에 넣어 output을 도출한 후, loss를 계산함
                    outputs = model(inputs)
                    loss = criterion(outputs, inputs)

                    # backward (optimize): training 단계에서만 수행
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # batch별 loss를 축적함
                running_loss += loss.item() * inputs.size(0)
                running_total += inputs.size(0)

            # epoch의 loss 및 accuracy 도출
            epoch_loss = running_loss / running_total

            if epoch == 0 or (epoch + 1) % 50 == 0:
                print('{} Loss: {:.4f}'.format(phase, epoch_loss))

            # validation 단계에서 validation loss가 감소할 때마다 best model 가중치를 업데이트함
            if phase == 'val' and epoch_loss < best_valid_loss:
                best_valid_loss = epoch_loss
                best_model_wts = copy.deepcopy(model.state_dict())

    # validation loss가 가장 낮았을 때의 best model 가중치를 불러와 best model을 구축함
    print('Best validation loss: {:4f}'.format(best_valid_loss))
    model.load_state_dict(best_model_wts)
    return model

In [ ]:
ae_dataloaders = {'train': ae_train_loader, 'val': ae_valid_train_loader}
ae_train_criterion = nn.MSELoss()
ae_model = train_model(ae_dataloaders, ae_model, ae_train_criterion, num_epochs, learning_rate, device)

### Step 4. 적합된 모델을 기반으로 train/test 데이터의 anomaly score 도출 (Test)

In [ ]:
# model 검증
def test_model(data_loader, model, criterion, device):
    model = model.to(device)
    
    model.eval()
    with torch.no_grad():
        test_loss = []
        for i, inputs in enumerate(data_loader):
            inputs = inputs[0].to(device)
            
            # Forward pass
            outputs = model(inputs)
            loss = criterion(outputs, inputs)
            loss = loss.reshape(loss.shape[0], -1).mean(axis=1)

            test_loss += list(loss.data.cpu().numpy())
    return test_loss

In [ ]:
# train/validation/test 데이터에 대한 최종 결과 도출
ae_test_criterion = nn.L1Loss(reduction='none')
ae_train_loader = torch.utils.data.DataLoader(ae_train_dataset, batch_size=batch_size, shuffle=False)

ae_train = test_model(ae_train_loader, ae_model, ae_test_criterion, device)
ae_valid = test_model(ae_valid_test_loader, ae_model, ae_test_criterion, device)
ae_test = test_model(ae_test_loader, ae_model, ae_test_criterion, device)

In [ ]:
# train/validation/test 데이터의 anomaly score 분포 시각화
fig, (ax1, ax2, ax3) = plt.subplots(nrows=3, ncols=1, figsize = (12, 8))

sns.distplot(ae_train, bins=100, kde=True, color='blue', ax=ax1)
sns.distplot(ae_valid, bins=100, kde=True, color='green', ax=ax2)
sns.distplot(ae_test, bins=100, kde=True, color='red', ax=ax3)
ax1.set_title("Train Data")
ax2.set_title("Validation Data")
ax3.set_title("Test Data")

### Step 5. Threshold 탐색

In [ ]:
# threshold 탐색
# score의 min ~ max 범위를 num_step개로 균등 분할한 threshold에 대하여 best threshold 탐색 
def search_best_threshold(score, y_true, num_step):
    best_f1 = 0
    best_threshold = None
    for threshold in np.linspace(min(score), max(score), num_step):
        y_pred = threshold < score

        f1 = f1_score(y_true, y_pred)
        if f1 > best_f1:
            best_f1 = f1
            best_threshold = threshold

    print('Best threshold: ', round(best_threshold, 4))
    print('Best F1 Score:', round(best_f1, 4))
    return best_threshold

In [ ]:
# best threshold 도출
ae_best_threshold = search_best_threshold(ae_valid, y_valid, num_step=1000)

In [ ]:
# 최종 결과 도출
ae_scores = pd.DataFrame(columns=['score', 'anomaly'])
for date, score in zip([X_train.index, X_valid.index, X_test.index], [ae_train, ae_valid, ae_test]):
    ae_score = pd.DataFrame(index=date)
    ae_score['score'] = score
    ae_score['anomaly'] = ae_best_threshold < score
    ae_scores = ae_scores.append(ae_score)

In [ ]:
ae_scores = ae_scores.sort_index()
ae_scores.head()

In [ ]:
# anomaly score plot 도출
def draw_plot(scores, threshold):
    normal_scores = scores[scores['anomaly'] == False]
    abnormal_scores = scores[scores['anomaly'] == True]

    plt.figure(figsize = (12,5))
    plt.scatter(normal_scores.index, normal_scores['score'], label='Normal', c='blue', s=3)
    plt.scatter(abnormal_scores.index, abnormal_scores['score'], label='Abnormal', c='red', s=3)
    
    plt.axhline(threshold, c='green', alpha=0.7)
    plt.axvline(data.index[int(len(data) * 0.5)], c='orange', ls='--')
    plt.axvline(data.index[int(len(data) * 0.7)], c='orange', ls='--')
    
    plt.xlabel('Date')
    plt.ylabel('Anomaly Score')
    plt.legend()
    
    plt.show()

In [ ]:
# 전체 데이터의 anomaly score 확인
draw_plot(ae_scores, ae_best_threshold)

In [ ]:
# FRR, FAR, F1 score, AUROC 도출
def calculate_metric(y_true, y_pred):
    # FRR, FAR
    cm = confusion_matrix(y_true, y_pred, labels=[True, False])
    tp, fn, fp, tn = cm.ravel()
    frr = fp / (fp + tn)
    far = fn / (fn + tp) 
    
    # F1 Score
    f1 = f1_score(y_true, y_pred)
    
    # AUROC, IE
    fpr, tpr, thresholds = roc_curve(y_true, y_pred, pos_label=1)
    auroc = auc(fpr, tpr)
        
    return frr, far, f1, auroc

In [ ]:
# 평가 지표 결과 확인
frr, far, f1, auroc = calculate_metric(y_test, ae_best_threshold < ae_test)

print("**  FRR: {}  |  FAR: {}  |  F1 Score: {}  |  AUROC: {}"
      .format(round(frr, 4), round(far, 4), round(f1, 4), round(auroc, 4)))

---------